Load data 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from data_preprocessing import *
from implementations import *
from validations import *
from helpers import *
from evaluation import *

base_path = os.path.dirname('data')
x_train_path = os.path.join(base_path, 'data', 'x_train.csv') #load the path of x_train
y_train_path = os.path.join(base_path, 'data', 'y_train.csv') #load the path of y_train
y,x,ind = load_data(x_train_path,y_train_path) #load x_train and y _train
#sub_sample true to test only first 5000 people

x_test_path = os.path.join(base_path, 'data', 'x_test.csv') #load the path of x_test
x_test, ids_test = load_test_data(x_test_path) #load x test

KeyboardInterrupt: 

Set values

In [3]:
# default values of gamma, lambda, degree and loss bias
gamma_ = 0.8
# lambda_ = 0.01 #uncomment to use lambda
degree = 2
loss_bias= 0.01

# search space for hyper-parameters
#create vector to find the best values of gamma, lambda, degree and loss biasgamma, lambda, degree and loss bias
degrees = [1, 2, 3]

#uncomment to use lambda
# lambdas = [
#     0.5,
#     0.1,
#     0.01,
#     0.001
# ]

gammas = [
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
]

loss_biases = [
    0.001,
    0.01,
    0.1
]

#set k_fold, max iteration and create a seed
k_fold = 5
seed = 381
max_iters = 5000
print(np.shape(x))
print(np.shape(x_test))

(328135, 320)
(109379, 320)


Validation

In [4]:
validation = False #set it True to do the cross-validation

if validation:
    cross_deg,f1_deg = cross_val_degree(degrees, k_fold, seed, x, y, gamma_, max_iters, loss_bias)

    print(cross_deg)
    print(f1_deg)
    #uncomment to do the cross-validation of the lambda
    # cross_deg = 2
    # cross_gamma = 0.8
    # cross_loss_bias= 0.01

    #uncomment to do the cross-validation of lambda
    # cross_lambda,f1_lamda = cross_val_lambda(lambdas, k_fold, seed, x, y, cross_deg, cross_gamma, max_iters, cross_loss_bias)

    # print(cross_lambda)
    # print(f1_lamda)

    cross_gamma,f1_gamma = cross_val_gamma_(gammas, k_fold, seed, x, y, cross_deg, max_iters, loss_bias)

    print(cross_gamma)
    print(f1_gamma)

    cross_loss_bias,f1_bias = cross_val_loss_bias(loss_biases, k_fold, seed, x, y, cross_deg, cross_gamma, max_iters)

    print(cross_loss_bias)
    print(f1_bias) 
else:
    #obtain the default values
    # cross_lambda = lambda_ #uncomment when the lambda is present
    cross_gamma = gamma_
    cross_deg = degree
    cross_loss_bias = loss_bias



Print the cross validation values

In [5]:
print(cross_deg)
# print(cross_lambda) #uncomment when the lambda is present
print(cross_gamma)
print(cross_loss_bias)

2
0.8
0.01


Preprocess

In [6]:
#preprocess the data
x_preproc, x_test_preproc,y_preproc = preprocess(x, cross_deg, True, True, True, x_test=x_test, y = y,repetition=6)
print(np.shape(x_preproc))

shape before poly (328135, 270)
shape after poly (328135, 541)
no of 0 299160
no of 1 28975
(473010, 541)
(473010,)
[1. 1. 1. ... 0. 1. 0.]
(473010, 541)


Reduce size of the x_train and y_train

In [7]:
#reduce the size of the matrix to do the logistic regression
x_preproc_train = x_preproc[0:100000,:]
y_preproc_train = y_preproc[0:100000]

print(np.shape(x_preproc_train))
print(np.shape(y_preproc_train))
print(np.shape(x_test_preproc))

(100000, 541)
(100000,)
(109379, 541)


Logistic regression

In [9]:
N,D= get_dim(x_test_preproc) #get the dimension of x_test_preproc
w0= np.zeros(D) #get a vector of shape D, the number of the columns of x_test_preproc

max_iters = 5000 #number of iteration of logistic regression

w,loss = logistic_regression(y_preproc_train, x_preproc_train, w0, max_iters, cross_gamma,cross_loss_bias) #Logistic regression
# w,loss = reg_logistic_regression_lasso(y_preproc_train, x_preproc_train, cross_lambda, w0, max_iters, cross_gamma, cross_loss_bias) #uncomment this function to do the regularized logistic regression with Lasso 

0 0.6931471805599436
100 0.5169657300431745
200 0.4961707196268774
300 0.47574477205049304
400 0.46898124056692725
500 0.46977040271981274
600 0.47351550776061985
700 0.462258585140022
800 0.462352218937216
900 0.464027728614034
1000 0.4544023633056885
1100 0.45750589945272435
1200 0.45887478553245126
1300 0.45695913773893887
1400 0.4543394918970676
1500 0.45816404327775456
1600 0.46174559569787293
1700 0.456069010111096
1800 0.46577772892618285
1900 0.45515932770845896
2000 0.4541808689209911
2100 0.46319213091931083
2200 0.45231710430546185
2300 0.45514048039946176
2400 0.4575297298665872
2500 0.4498926979737815
2600 0.4545715805188148
2700 0.4520351928441213
2800 0.4518630130077457
2900 0.45954788717799366
3000 0.4583120972083178
3100 0.44974017431647534
3200 0.4498697987768877
3300 0.46161266631821635
3400 0.4606516632840727
3500 0.4486197174062137
3600 0.4505910455443533
3700 0.44910277302253215
3800 0.4586416387893244
3900 0.45812059317964804
4000 0.46024718470363235
4100 0.45081

Logistic prediction

In [10]:
y_hat= logistic_prediction(x_test_preproc,w) #logistic prediction
print(y_hat)
y_predictions = zero_to_minone(y_hat) #change from 0 to -1

print(y_predictions)

[0.3931411  0.00679339 0.20048128 ... 0.13262211 0.82095147 0.24946657]
[-1. -1. -1. ... -1.  1. -1.]


Submission

In [11]:
create_csv_submission(ids_test,y_predictions, "submission_exp3E.csv") #create submission

See f1 score of prediction on some train set

In [12]:
#test the model on the last 50.000 subject of the train dataset
try_test = x_preproc[-50000:,:]
try_y_preproc = y_preproc[-50000:]

try_y_hat = logistic_prediction(try_test,w)

try_y_predictions = zero_to_minone(try_y_hat)

f1_score(try_y_preproc,try_y_predictions)

Accuracy:  78.318 %
Percentage of correct -1: 84.60419368101458
Percentage of correct 1: 67.50448833034112
F1 score: 0.6959643267801554


0.6959643267801554